In [1]:
# ! gdown 1qhn9-RgkfFHIGD7pDGIvwjQehGKM-5Bi

In [2]:
from textblob import TextBlob

In [3]:
import pandas as pd
df = pd.read_csv('./scraping/doge.csv')

In [4]:
df

,Datetime,Tweet Id,Text,Username,Followers
0,1.614557e+09,1366176256534974464,We made @CoinDesk again.\nAnd it's good news!\...,JacksonBollock2,441
1,1.614557e+09,1366176192529723403,@DogeCoinCEO_ Make a meme that shows Elon’s Ru...,Mineo4prez,5
2,1.614557e+09,1366176181171548162,Live now! Money giveaway #dogecoin #doge http...,bluehairdave,164
3,1.614557e+09,1366176162733555714,@Mar3issa \nAfter tomorrow #dogecoin dm for de...,MCDoge_1,15674
4,1.614557e+09,1366175961880879119,#Dogecoin https://t.co/aveFCeJvTF,ganggreenie36,2680
...,...,...,...,...,...
968780,1.613938e+09,1363581226116907008,@Click2vicky @BrandonDrowns3 @elonmusk 'This i...,RamadanMagdy15,2
968781,1.613938e+09,1363581218898522113,@Click2vicky @MuratMmturk @elonmusk Announcing...,bumiserpongbsde,127
968782,1.613938e+09,1363581217782837250,@Click2vicky @PeterSchiff @elonmusk My mum jus...,Mdshahn41660727,2
968783,1.613938e+09,1363581217325674496,@Click2vicky @unusual_whales @elonmusk One of ...,MA1978250837,9


In [5]:
import re
import nltk
from nltk import PorterStemmer
ps = PorterStemmer()
nltk.download("stopwords")
from nltk.corpus import stopwords
stop = stopwords.words('english')
def cleantweets(txt):
  # Remove mentions
  txt = re.sub(r"@","", txt)
  # Remove hashtags
  txt = re.sub(r"#","", txt)
  # Remove urls
  txt = re.sub(r"https?:\/\/[ A−Za−z0 −9\.\/]+", "", txt)
  # Remove Stop Words
  txt = ' '.join([word for word in txt.split() if word not in (stop)])
  #Stemming
  txt = ' '.join([ps.stem(word) for word in txt.split()])
  return txt

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/piyushk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def getsubjectivity(txt):
  return TextBlob(txt).sentiment.subjectivity

In [7]:
def getpolarity(txt):
  return TextBlob(txt).sentiment.polarity

In [8]:
def getanalysis(a):
  if a < 0:
    return "Negative"
  elif a == 0:
    return "Neutral"
  else:
    return "Positive"

In [9]:
def prepare_twitter_data(tweet_data):
  results = tweet_data.copy()
  results["clean text"] = results["Text"].apply(cleantweets)
  results["polarity"] = results["Text"].apply(getpolarity)
  results["subjectivity"] = results["Text"].apply(getsubjectivity)
  results["tweet_class"] = results["polarity"].apply(getanalysis)
  return results

In [11]:
processed_tweets = prepare_twitter_data(df)

In [12]:
from langdetect import detect

df = pd.DataFrame(
    columns=[
        "Datetime",
        "Tweet Id",
        "Text",
        "Username",
        "Followers",
        "clean text",
        "polarity",
        "subjectivity",
        "tweet_class",
    ]
)
from contextlib import suppress
import multiprocessing

def filter_en_text(i, x):
    with suppress(Exception):
        return detect(x) == "en"
with multiprocessing.Pool() as pool:
    # Use the pool to apply the filter function to each element in parallel
    results = pool.starmap(filter_en_text, enumerate(processed_tweets["clean text"].values))
    # Filter the original DataFrame using the results
    results = [False if v is None else v for v in results]
    df = processed_tweets[results]

In [14]:
processed_tweets.to_csv('[doge] processed_tweets.csv')